In [1]:
import numpy as np
import math
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.translate.metrics import alignment_error_rate
from nltk.metrics.scores import (precision, recall)
from operator import itemgetter

In [2]:
#DATA
file = open('./train.en', 'r')
train_en = file.read()
raw_sentences_train_en = train_en.split("\n")

file = open('./train.hi', 'r')
train_hi = file.read()
sentences_train_hi = train_hi.split("\n")

file = open('./test.en', 'r')
test_en = file.read()
raw_sentences_test_en = test_en.split("\n")

file = open('./test.hi', 'r')
test_hi = file.read()
sentences_test_hi = test_hi.split("\n")

file = open('./dev.en.txt', 'r')
dev_en = file.read()
raw_sentences_dev_en = dev_en.split("\n")

file = open('./dev.hi', 'r')
dev_hi = file.read()
sentences_dev_hi = dev_hi.split("\n")

In [3]:
def make_lower_case(data):
    list_ = []
    for sentence in data:
        list_.append(sentence.lower())
    return list_

In [4]:
sentences_dev_en = make_lower_case(raw_sentences_dev_en)
sentences_test_en = make_lower_case(raw_sentences_test_en)
sentences_train_en = make_lower_case(raw_sentences_train_en)

In [5]:
def is_converged(new, old, epoch):
    epsilone = 0.00000001
#     new = list(new.values())
#     old = list(old.values())
    
#     new_val = []
#     for (hin, ddict) in new.items():
#         for (eng, prob) in ddict.items():
#             new_val.append('{0:.8f}'.format(prob))
            
#     old_val = []
#     for (hin, ddict) in old.items():
#         for (eng, prob) in ddict.items():
#             old_val.append('{0:.8f}'.format(prob))        

#     for i in range(len(old_val)):
#         if math.fabs(float(new_val[i]) - float(old_val[i])) > epsilone: 
    if epoch < 15:
        return False
    return True

In [10]:
def perform_EM(en_sentences, hi_sentences):
    
    uni_ini = 0.00001
    
    translation_prob = defaultdict(lambda: float(uni_ini))
    translation_prob_prev = defaultdict(float)
    
    epoch = 0
    
    while True:

        epoch += 1
        print("epoch num:", epoch,"\n")
        count = defaultdict(float)
        total = defaultdict(float)
        for index_sen, hin_sen in enumerate(hi_sentences):
            #compute normalization
            hin_sen_words = hin_sen.split(" ")
            s_total = defaultdict(float)
            for hin_word in hin_sen_words:
                s_total[hin_word] = 0
                eng_sen_words = en_sentences[index_sen].split(" ")
                for eng_word in eng_sen_words:
                        s_total[hin_word] += translation_prob[(hin_word, eng_word)]
            
            #collect counts
            for hin_word in hin_sen_words:
                eng_sen_words = en_sentences[index_sen].split(" ")
                for eng_word in eng_sen_words:
                        count[(hin_word, eng_word)] += translation_prob[(hin_word, eng_word)]/s_total[hin_word]
                        total[eng_word] += translation_prob[(hin_word, eng_word)]/s_total[hin_word]                   

        #estimate probabilities
        for (hin_word, eng_word) in translation_prob.keys():
                translation_prob[(hin_word, eng_word)] = count[(hin_word, eng_word)]/total[eng_word]

        if is_converged(translation_prob, translation_prob_prev, epoch) == True:
            break
        
        translation_prob_prev = translation_prob
        
        
    return translation_prob


In [7]:
def train_model(sentences_train_en, sentences_train_hi):
    
    translation_prob = perform_EM(sentences_train_en, sentences_train_hi)
    return translation_prob


In [11]:
tef = train_model(sentences_train_en, sentences_train_hi)

epoch num: 1 

epoch num: 2 

epoch num: 3 

epoch num: 4 

epoch num: 5 

epoch num: 6 

epoch num: 7 

epoch num: 8 

epoch num: 9 

epoch num: 10 

epoch num: 11 

epoch num: 12 

epoch num: 13 

epoch num: 14 

epoch num: 15 



In [ ]:
# np.save("./models/IBMmodel1tef_3", tef)

In [14]:
# iterations = 0
# for ((e_word, hin_word), value) in sorted(tef.items(), key=itemgetter(1), reverse=True):
#     if iterations < 100:
#         print("{:<20}{:>20.2}".format("t(%s|%s)" %(e_word, hin_word), value))
#     else:
#         break
#     iterations += 1

In [15]:
print(tef[('तथा','and')])

0.11966619980620694


In [19]:
predicted_translations = test_model(sentences_dev_hi, tef)

hin: बल्कि आप नेत्ररोगों से भी बचे रहेंगे ।


AttributeError: 'float' object has no attribute 'values'

In [11]:
def find_recall_precision(predicted_translations):
    y_pred = list(predicted_translations)
    y_true = sentences_dev_hi
    recall = recall(y_true, y_pred, average=None)
    precision = precision(y_true, y_pred, average=None)
    return recall, precision;